In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_metric
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer

In [2]:
df = pd.read_csv('./data/daum_movie_review.csv')
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [3]:
y = [0 if rate < 6 else 1 for rate in df.rating]

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(df.review.tolist(), y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

In [5]:
metric = load_metric('accuracy')

C:\Users\admin\AppData\Local\Temp\ipykernel_8328\2545647748.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [6]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions, references=labels)

In [7]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [8]:
sentence = '안녕하세요. 반갑습니다.'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
inputs = tokenizer(sentence)
print(tokenizer.tokenize(sentence), inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.'] {'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [9]:
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [10]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_train)
test_dataset = OurDataset(test_input, y_train)

In [11]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [12]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=1,
                                  evaluation_strategy='steps',
                                  eval_steps=100,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  warmup_steps=200,
                                  weight_decay=0.01)
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset,
                  compute_metrics=compute_metrics)

In [13]:
trainer.train()

c:\Users\admin\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8282
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 518
  Number of trainable parameters = 177854978


  0%|          | 0/518 [00:00<?, ?it/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_8328\475304883.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')

In [ ]:
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [ ]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_train)
test_dataset = OurDataset(test_input, y_train)

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=16)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=16)

In [ ]:
class MyModel(nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model
        self.classifier = nn.Linear(self.token_size, self.num_labels)
    
    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs)
        bert_clf_token = outputs.last_hidden_state[:, 0, :]
        return self.classifier(bert_clf_token)

In [ ]:
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size)